# Intro to Prompt Engineering with LLMs

[This course](https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/) is produced by the ubiquitous Andrew Ng & OpenAI Engineer Isa Fulford.

The course starts with a distinction in the functionality between 2 types of LLM:

**Base LLM**: Goos at predicting the next predicted word, so can exhibit autocomplete
behaviour. 

**Instruction Tuned LLM**: A base LLM trained to respond to instructions such as 
questions. This often involves **RLHF** - reinforcement learning with human feedback. 

The LLM output should be:

**H**elpful, **H**onest & **H**armless

## Prompt Engineering

To follow along, you'll need:
* to pip install `openai` - this is in the requirements.txt.
* to generate an api key from the [openai website](https://platform.openai.com/account/api-keys).


In [20]:
import toml
import os
from pyprojroot import here
import openai

In [26]:
CONFIG = toml.load(os.path.join(here(), ".ignore_me", "secrets.toml"))
PROMPT_ENGINEER = CONFIG["openai"]["PROMPT_ENGINEER"]
openai.api_key = PROMPT_ENGINEER


In [32]:
def get_answer(usr_input, mod="gpt-3.5-turbo"):
    """Get an answer from chat GPT.

    Args:
        usr_input (str): A prompt generated by the user.
        mod (str, optional): The model of GPT to use. Defaults to "gpt-3.5-turbo".

    Returns:
        str: The model response text.
    """
    mod_input = [{"role": "user", "content": usr_input}]
    resp = openai.ChatCompletion.create(
        model=mod,
        messages=mod_input,
        temperature=0 # degree of randomness 
    )
    return resp.choices[0].message["content"]

The above is not working as rate limits asssociated with my tier of key.

## Guidance on Prompts

### Use delimeters

Longer, more specific prompts are specified. You can use delimeters such as:

* `"""`
* ` ``` `
* `---`
* `<>`
* `<tag></tag>` (XML-flavoured tags)

Then use the available tags to stucture your prompt as follows:


In [34]:
war_n_peace  = """
Some scintillating text tht you wint to summarise  / analyse / do something with \
enburing enouhg context is provided.
"""
# now for the instruction
usr_input = f"""Correct any typos in the text delimited by triple backticks\
    ```{war_n_peace}````
"""
# get_answer(usr_input)

**Response generated**:
'Some scintillating text that you want to summarise / analyse / do something with ensuring enough context is provided.'

This is also a good defence against prompt injection. By exposing the `war_n_peace` input
only to the user, chatGPT will be summarising these dynamic inputs rather than following
any instructions contained within.

### Specify structured output

You can ask for HTML, XML, JSON etc, provide enough info for the model to cope though:


In [35]:
prompt = """
Make up 3 new science fiction movies, including their director & tagline. Return
format should be JSON with the following keys: movie_name, director_name, tagline.
"""
# get_answer(prompt)

**Response generated**:

'{\n  "movie1": {\n    "movie_name": "Galactic Odyssey",\n    "director_name": "Christopher Nolan",\n    "tagline": "Journey beyond the stars"\n  },\n  "movie2": {\n    "movie_name": "Chrono Shift",\n    "director_name": "Denis Villeneuve",\n    "tagline": "Time is not what it seems"\n  },\n  "movie3": {\n    "movie_name": "Quantum Paradox",\n    "director_name": "James Cameron",\n    "tagline": "The laws of physics are about to be broken"\n  }\n}'

### Check conditions

You can ask the model to check some assumptions prior to making a response. For example:

In [ ]:
txt = "soundgarden, queens of the stone age, mark lanegan band"

prompt = f""" 
Check whether some well-known rock band names are provided within the \
    backticked-delimiited text:
    ```{txt}```
    If band names are found, generate a new album name for each band found.
    If no band was found, return \"No known bands found.\" 
"""
# get_answer(prompt)

**Response generated**:

'New album names:\n- Soundgarden: "Echoes of the Black Hole"\n- Queens of the Stone Age: "Villains of the Valley"\n- Mark Lanegan Band: "Phantom Radio Sessions"\n\nNote: As an AI language model, I cannot determine whether these album names already exist or not.'

In [39]:
txt = "eggs, ham"
#  get_answer(prompt)
# response: 'No known bands found.'

### Few-shot prompting

This is when you provide the model with an exemplar answer. Let's take a look:

In [36]:
prompt = """
I need you to answer in the style of Yoda:

<Luke>: Why can't I lift these stones with the force?
<Yoda>: Work hard you must. Easy, it is not.
<Luke>: I'm hungry. Can you fix me up a womp rat stew?
"""

# get_answer(prompt)

**Response generated**:  
'`<Yoda>`: Hungry, are you? Womp rat stew, I can make. But first, patience you must have. Train your mind and body, you must, to become a Jedi.'

## Providing Think Time

Asking the model to take longer time can ensure it provides the correct answer rather
than quickly returning an incorrect answer. Also, breaking down complex tasks into 
separate instructions:

In [37]:
txt = """"
Der Strauß, den ich gepflücket,
Grüße dich vieltausendmal!
Ich hab mich oft gebücket,
Ach, wohl eintausendmal,
Und ihn ans Herz gedrücket
Wie hunderttausendmal!.
"""
prompt = f"""
Perform the following instructions, seperate your answers with carriage returns:
1. Translate the backtick-delimited text into American English: ```{txt}```
2. Translate the same backtick-delimited text into United Kingdom English.
3. Underline the text generated in steps 2 and 3 wherever they differ.
4. Summarise the American English version of the text to be as succinct as possible.

"""
# get_answer(prompt)

**Response generated:**

'1. "The bouquet that I picked, greets you a thousand times! I have often bent down, oh, probably a thousand times, and pressed it to my heart like a hundred thousand times!"\n2. "The bouquet that I picked, greets you a thousand times! I have often bent down, oh, probably a thousand times, and pressed it to my heart like a hundred thousand times!"\n3. No differences found.\n4. The speaker greets the recipient with a bouquet that they have picked and expresses how many times they have held it close to their heart.'